In [1]:
import numpy as np
import scipy.constants as const

In [83]:
# constants

# D2 transition angular frequency [Hz]
F_RES = 2 * const.pi * const.c / 670.977338e-9

# 780 nm to angular frequency [Hz]
F_BRAGG = 2 * const.pi * const.c / 780e-9

# bragg power [W]
P_BRAGG = 10e-3

# mass of lithium [kg]
M_LI = 9.9883414e-27

# beam size on atoms [m]
W_SIZE = 50e-6

# D2 transition line width [Hz]
GAMMA = 2 * const.pi * 5.8724e6

# detuning of bragg beam wrt. the resonant beam
DELTA = np.abs(F_BRAGG - F_RES)


In [84]:
def intensity(power, waist):
    """intensity [W m^-2]"""

    return 2 * power / (const.pi * waist**2)

In [85]:
def U_dip(power, waist):
    """dipole potential [J]"""

    return 3 * const.pi * const.c**2 / (2 * F_RES**3) * GAMMA / DELTA * intensity(power, waist)

In [86]:
def trap_freq(power, waist):
    """trap frequency [Hz]"""

    return np.sqrt(4 * U_dip(power, waist) / (M_LI * waist**2)) / (2 * const.pi)

In [87]:
trap_freq(P_BRAGG, W_SIZE)

136.3785226168286